# Анализ данных по компании "Газпром"

## Чтение данных, которые были распаршены.

In [ ]:
import pandas as pd

official_website = pd.read_csv('data/gazprom.csv', encoding='cp1251')

news_1 = pd.read_csv('data/kommersant.csv', encoding='cp1251')
news_2 = pd.read_csv('data/rbc.csv', encoding='cp1251')

other = pd.read_csv('data/raexpert.csv', encoding='cp1251')

## Просуммаризируем новости о компании:

Инициализация transformer-based pre-train модели суммаризации, адаптированной на корпусе текстов новостей.

reference: https://deeppavlov.ai

In [140]:
from transformers import MBartTokenizer, MBartForConditionalGeneration

model_name = "IlyaGusev/mbart_ru_sum_gazeta"
tokenizer_s = MBartTokenizer.from_pretrained(model_name)
model_s = MBartForConditionalGeneration.from_pretrained(model_name)

def get_summary(input_text, token_qnt = 800):
    article_text = input_text

    input_ids = tokenizer_s(
        [article_text],
        max_length=token_qnt,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )["input_ids"]

    output_ids = model_s.generate(
        input_ids=input_ids,
        no_repeat_ngram_size=4
    )[0]

    summary = tokenizer_s.decode(output_ids, skip_special_tokens=True)
    return summary

Демонстрация работы модели суммаризации на примере текста одной из новостей:

In [53]:
# исходный текст новости
news_1.iloc[0]['texts']

'По данным “Ъ”, крупнейший строительный подрядчик «Газпрома» «Газстройпром» профинансировал покупку третьей стороной «Газпром бурения» почти на 68 млрд руб. «Газпром бурение» — основной профильный подрядчик «Газпрома», его выручка за 2021 год составила около 101 млрд руб., чистая прибыль — около 7 млрд руб.Монополия продала «Газпром бурение» Аркадию Ротенбергу в 2011 году за 4 млрд руб. Попав под санкции в 2014 году, бизнесмен передал долю сыну Игорю. Последний вышел из капитала «Газпром бурения» в прошлом году: в августе 2021 года Игорь Ротенберг (доля 78,7%), ЗПИФ «Оскар» (16,3%) и Александр Замятин продали компанию неизвестному АО «НПС Стар» (бенефициаром значилась Екатерина Михайлова), а буквально через месяц 100% в «Газпром бурении» перешли к АО «Фирма Армадис» с тем же акционером.«Фирма Армадис» расплатилась с «НПС Стар», в ее отчетности отражены «прочий доход» на чуть менее 67,929 млрд руб., а также расходы в 67,97 млрд руб. и убыток в 8 млн руб. Таким образом «Газстройпром» пом

In [54]:
# суммаризация текста, представленного выше
get_summary(news_1.iloc[0]['texts'])

'«Газстройпром» профинансировал покупку третьей стороной «Газпром бурения» почти на 68 млрд руб. По данным «Ъ», крупнейший строительный подрядчик «Газпрома» помог расплатиться за актив с Аркадием Ротенбергом. Смена владельцев потребовалась после того, как бизнесмены попали под жесткие санкции США, тогда как их компании получали подряды на крупнейшие стройки «Газпрома», в том числе в Европе.'

Генерация суммаризаций для всех текстов из корпуса news_1

In [30]:
a = time.time()
news_1_summary = []
for i in range(news_1.shape[0]):
    cur_summary = get_summary(news_1.iloc[i]['texts'])
    news_1_summary.append(cur_summary)
print(time.time() - a)

521.2268605232239


In [38]:
full_news_1_summary = ' <s> '.join(news_1_summary)

Пример Генерации менее детального обобщения на основе уже суммаризированных текстов. (уровень детализации можно повысить - повысив число частей)

In [50]:
news_1_part_1 = ' <s> '.join(news_1_summary[:10])
news_1_part_2 = ' <s> '.join(news_1_summary[10:20])
news_1_part_3 = ' <s> '.join(news_1_summary[20:30])
news_1_part_4 = ' <s> '.join(news_1_summary[30:40])

In [51]:
news_1_part_1_summary = get_summary(news_1_part_1, token_qnt=1000)
news_1_part_2_summary = get_summary(news_1_part_2, token_qnt=1000)
news_1_part_3_summary = get_summary(news_1_part_3, token_qnt=1000)
news_1_part_4_summary = get_summary(news_1_part_4, token_qnt=1000)

In [64]:
news_1_meta_summary = '\n'.join([news_1_part_1_summary, news_1_part_2_summary, news_1_part_3_summary, news_1_part_4_summary])
print(news_1_meta_summary)

«Газпром» может потерять хранилище Haidach в Зальцбурге из-за санкций США, которые уже ударили по «Роснефти» и «Южному потоку». По мнению экспертов, это может быть связано с тем, что Польша уже давно планировала прекратить прямые поставки российского газа в Европу и готовится к судебным разбирательствам в Стокгольмском арбитраже.
«Газпром» прекратил поставки газа в Польшу из-за отказа европейских компаний перейти на новую схему оплаты российского газа в рублях. Эксперты полагают, что раскол на европейском рынке газа «уже обозначился» и будет нарастать по мере поляризации позиций Gazprom.
«Газпром» и CNPC подписали новый долгосрочный контракт на поставку газа в Китай. Накануне зимы объемы закачки в европейские ПХГ находились на критически низких уровнях, сейчас этот показатель опустился до 36,2%. В «Роснефти» считают, что это «дополнительный вклад в рост цен».
«Газпром» выкупил западный участок газопровода «Южный поток» за 1,3 млрд руб., несмотря на то, что Кишинев не исключил, что это 

Оборачиваем необходимый функционал для генерации так называемых "meta-summary" в функцию:

на вход передается массив суммаризаций исходных текстов и желаемый уровень детализации - чем больше число, тем более подробным получается суммаризация

In [84]:
from math import floor

def meta_summary(list_of_summaries, depth):
    chunk_size = floor(len(list_of_summaries) / depth)
    
    chunk_summary_storage = []
    
    for i in range(depth):
        cur_storage = []
        
        tmp = chunk_size
        if i == (depth - 1):
            tmp += len(list_of_summaries) - depth * chunk_size
        for j in range(tmp):
            cur_storage.append(list_of_summaries[(i * chunk_size) + j])
            
        cur_text_to_summarize = ' <s> '.join(cur_storage)
        cur_chunk_summary = get_summary(cur_text_to_summarize, token_qnt=1000)
        chunk_summary_storage.append(cur_chunk_summary)
        
    meta_sum = '\n'.join(chunk_summary_storage)
    return meta_sum

In [86]:
news_1_meta_summary = meta_summary(news_1_summary, 7)

In [89]:
# Пример получившейся сводки, сгенерированной на основе текстов нескольких десятков новостей:
print(news_1_meta_summary)

«Газстройпром» профинансировал покупку третьей стороной «Газпром бурения» почти на 68 млрд руб. Смена владельцев потребовалась после того, как бизнесмены попали под жесткие санкции США, тогда как их компании получали подряды на крупнейшие стройки «Газпрома», в том числе в Европе.
«Газпром» не сможет использовать трубопровод «Ямал—Европа» для прокачки газа в Европу из-за проблем с поставками газа через Украину. На фоне стремительного ослабления курса рубля инвесторы не исключают срабатывания большого количества стоп-заявок. Глава «Роснефти» Игорь Сечин предлагает установить для монополии минимальный и максимальный объем продаж газа на бирже СПбМТСБ.
«Газпром» прекратил поставки газа реверсом в Польшу, несмотря на то, что Германия прекратила физический реверс по Ямалу—Европе. Это произошло из-за появившихся заявок потребителей Италии и Франции, которые обеспечиваются по польскому коридору. Эксперты полагают, что раскол на европейском рынке газа «уже обозначился» и будет нарастать по мере

Теперь сравним это с другим способом получения сводки - базовым прочтением заголовков:

In [90]:
text_headings_storage = [news_1.iloc[i]['articles'] for i in range(len(news_1))]
print('\n'.join(text_headings_storage))

«Газпром» вложился в бурение
«Газпром» прекратит поставки Shell в Германии
Кому меньше всех надо
Запасай, или потеряешь
«Газпром» увеличил поставки газа в Европу
Спекулянты отыгрались на «Газпроме»
Газу перекрывают маршруты
«Газпром» призывают к биржевой дисциплине
Газ меняет направление
Боррель заявил об отсутствии планов ЕС отказаться от поставок российского газа
«Газпром»: Германия перестала поставлять российский газ в Польшу
«Газпром»: экспорт газа в Китай с начала года вырос почти на 60%
«Газпром» успел заработать в 2021 году
«Газпром» планирует в 2022 году снизить добычу газа на 4%
Союзные газударства
«"Газпром" практически ничего не теряет»
Газ запутался в платежах
Германия берет «Газпром» на себя
«Газпром» рубит зарубежные связи
Bloomberg: Великобритания планирует временно национализировать «дочку» «Газпрома»
Газ отклонился от генераторной линии
«Газпром» не стал бронировать мощности Ямал—Европа до октября
Атомные метанморфозы
В газовой сделке запахло Сахалином
«Газпром» подпис

Итак, видим, что заголовки:
- СМИ часто выбирают громкие заголовки, не несущие смысловой нагрузки - например: "Германия берет «Газпром» на себя", "Газ в чрезвычайном режиме"
- СМИ нередко выбирают заголовки, связанные с новостью лишь косвенно - например: "Союзные газударства"
- Часто, даже при условии корреляции заголовка с реальной повесткой, никакой полезной информации извлечь из него не получится, непонятен контекст новости - например: "«Газпром» не стал бронировать мощности Ямал—Европа до октября"
- Часть заголовков дублируют друг друга (в нашем решении дублирование сведено к минимуму) - например: "«Газпром» снова не забронировал мощности газопровода Ямал—Европа" и "«Газпром» вновь не забронировал мощности газопровода Ямал—Европа"

Проведем аналогичные манипуляция со вторым корпусом новостей news_2:

In [92]:
news_2_summary = []
for i in range(news_2.shape[0]):
    cur_summary = get_summary(news_2.iloc[i]['texts'])
    news_2_summary.append(cur_summary)
print(time.time() - a)

7131.060654401779


In [94]:
news_2_meta_summary = meta_summary(news_2_summary, 7)

In [96]:
print(news_2_meta_summary)

В Санкт-Петербурге открыли памятник «Петр I, спасающий утопающих близ Лахты» по мотивам памятника Леопольда Бернштама. Ранее Россия объявила о новом механизме оплаты газа, из-за которого страна лишилась доступа примерно к половине золотовалютных резервов. Ранее Shell заявила о постепенном отказе от закупок российской нефти, а также нефтепродуктов, трубопроводного газа и СПГ еще в начале марта.
«Газпром» приостановил поставки газа в Молдавию из-за неуплаты за апрель. Накануне компания GasTerra объявила, что не будет выполнять требования «Газпрома» о переходе на новую схему оплаты топлива. Эксперты считают, что это может привести к нарушению введенных Евросоюзом санкций, а также повлечь за собой другие финансовые и операционные риски.
Поставки российского газа в Молдавию могут быть прекращены из-за того, что компания GasTerra не будет платить за топливо в рублях. Ранее экс-канцлер Германии Герхард Шрёдер отклонил предложение войти в совет директоров «Газпрома».
В совет директоров «Газпро

## Теперь попробуем оценить семантическую окраску новостей про Газпром:

Инициализация модели оценки семантической окраски текстов:

In [113]:
import torch
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment-rusentiment')
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment-rusentiment', return_dict=True)

@torch.no_grad()
def predict_sentiment(text):
    inputs = tokenizer(text, max_length=2048, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.argmax(predicted, dim=1).numpy()
    return predicted

Демонстрация работы модели:

In [114]:
print(predict_sentiment('у Газпрома все плохо'))

[2]


In [101]:
print(predict_sentiment('у Газпрома все хорошо'))

[1]


In [102]:
print(predict_sentiment('у Газпрома все нормально'))

[0]


In [124]:
predict_sentiment(news_1_summary)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

И в самом начале сталкиваемся с проблемой: тексты изданий, избранных для анализа, написаны сухим и деловым языком, для работы с которым модели необходим fine-tuning (адаптация).
К сожалению, ресурсы необходимые для fine-tuning, выходят за рамки данного проекта, однако это одно из направлений для развития.

## Перейдем к информации с официального сайта компании и информации с сайта рейтингово агенства "Эксперт РА":

In [125]:
official_website

,texts,articles
0,РЕЛИЗСегодня в Санкт-Петербурге состоялись тор...,«Газпром» поздравил Санкт-Петербург с 350-лети...
1,РЕЛИЗСегодня в Москве под руководством Председ...,Под руководством Виктора Зубкова прошел Форум ...
2,"РЕЛИЗВ январе-мае 2022 года «Газпром», по пред...",Добыча и поставки газа: итоги пяти месяцев
3,РЕЛИЗПо состоянию на конец рабочего дня 31 мая...,«Газпром» полностью остановил поставку газа ко...
4,РЕЛИЗПо состоянию на конец рабочего дня 31 мая...,«Газпром» полностью остановил поставку газа ко...


In [138]:
other

,texts,articles
0,ПАО «Газпром» – глобальная энергетическая комп...,«Эксперт РА» подтвердил рейтинг компании «Газп...
1,Группа Газпром (далее – Группа) - одна из круп...,«Эксперт РА» присвоил рейтинг компании «ГАЗПРО...
2,ПАО «Газпром» – глобальная энергетическая комп...,«Эксперт РА» присвоил рейтинг компании «Газпро...
3,"Согласно исследованию, в составе 200 крупнейши...",«Эксперт 400»: для роста капитализации российс...


## Описание компании можно сократить, выделив самое важное с помощью суммаризации:

In [143]:
official_website_summary = []
for i in range(official_website.shape[0]):
    cur_summary = get_summary(official_website.iloc[i]['texts'])
    official_website_summary.append(cur_summary)

In [144]:
other_summary = []
for i in range(other.shape[0]):
    cur_summary = get_summary(other.iloc[i]['texts'])
    other_summary.append(cur_summary)

#### Замечаем, что в части текстов суммаризаций модель не выделила название компании, как ключевое, будем трактовать такие новости как несущественные и не включим их в источники для генерации overview:

In [177]:
import re
official_website_summary_filtered = list(filter(lambda text: re.search(r'^(.*)([Гг]азпром)|([Gg]azprom)(.*)$', text), official_website_summary))
other_summary_filtered = list(filter(lambda text: re.search(r'^(.*)([Гг]азпром)|([Gg]azprom)(.*)$', text), other_summary))

In [183]:
official_website_overview = '\n'.join(official_website_summary_filtered)
other_overview = '\n'.join(other_summary_filtered)

#### Полученные overview Газпрома на основе официального сайта и сайта рейтингового агенства:

In [187]:
print(official_website_overview)

В Москве состоялся Форум «Технологическая независимость сегодня — мировое лидерство завтра». На мероприятии приняли участие представители Министерства промышленности и торговли РФ, ПАО «Газпром» и других энергетических компаний, руководители промышленных предприятий — членов Ассоциации производителей оборудования «Новые технологии газовой отрасли».
«Газпром» в январе-мае добыл меньше газа, чем в прошлом году, на 4,8%. Поставки компании из газотранспортной системы на внутренний рынок находятся на уровне прошлого года. На этом фоне поставки газа российским потребителям по Единой системе газоснабжения 22 раза достигали рекордных суточных показателей за 10 лет.
«Газпром экспорт» не получил платеж за поставки газа в апреле от компании Shell Energy Europe Limited в соответствии с Указом Президента РФ No172 от 31.03.2022 г.Платежи за газ, поставляемый с 1 апреля, должны осуществляться в рублях с использованием новых реквизитов, о чем контрагенты были своевременно проинформированы.
«Газпром эк

In [186]:
print(other_overview)

Агентство Эксперт РА опубликовало результаты внутреннего аудита ПАО «Газпром» за 2018 год. Эксперты отмечают, что в компании сохраняется высокая степень защиты прав собственников и остальных стейкхолдеров, в том числе в АО «СОГАЗ». При этом сдерживающее влияние на уровень рейтинга оказывает невысокая доля независимых директоров – 27,3% (3 независимых директора из 11). Агентство отмечает появление в компании новой должности - старший независимый директор в рамках развития взаимодействия между независимыми директорами, акционерами и Председателем Совета директоров.
Группа Газпром — одна из крупнейших в мире энергетических компаний, крупнейший мировой производитель газа. Головной компанией Группы является ПАО «Газпром» и является одним из крупнейших налогоплательщиков Российской Федерации. Агентство оценивает устойчивость к внешним шокам отрасли Группы как умеренную. Это обусловлено относительно стабильным спросом на углеводороды, низкой себестоимостью добычи, вертикально-интегрированной 

## И, наконец, попробуем выделить сущности в наших корпусах:

Для выделения сущностей в текстах, воспользуемся моделью, решающую задачу NER (Named entity recognition):
    
reference: https://spacy.io

In [212]:
import spacy
from spacy.lang.ru.examples import sentences 

ner_model = spacy.load("ru_core_news_sm")

In [235]:
official_website_texts = ' '.join([official_website.iloc[i]['texts'] for i in range(official_website.shape[0])])
parsed_other = ner_model(official_website_texts)

freq_dict = dict()
for token in parsed_other:
    if token.pos_ == 'PROPN':
        if token.lemma_ in freq_dict:
            freq_dict[token.lemma_] += 1
        else:
            freq_dict[token.lemma_] = 1
sorted_ners_other = sorted(freq_dict.items(), key=lambda kv: -kv[1])
sorted_ners_other[:10]

[('пётр', 13),
 ('газпром', 12),
 ('м', 7),
 ('петербург', 5),
 ('александр', 5),
 ('пао', 4),
 ('санкт', 3),
 ('-', 3),
 ('алексей', 3),
 ('миллер', 3)]

In [238]:
other_texts = ' '.join([other.iloc[i]['texts'] for i in range(other.shape[0])])
parsed_official_website = ner_model(other_texts)

freq_dict = dict()
for token in parsed_official_website:
    if token.pos_ == 'PROPN':
        if token.lemma_ in freq_dict:
            freq_dict[token.lemma_] += 1
        else:
            freq_dict[token.lemma_] = 1
sorted_ners_official_website = sorted(freq_dict.items(), key=lambda kv: -kv[1])
sorted_ners_official_website[:10]

[('группа', 23),
 ('газпром', 16),
 ('совет', 15),
 ('россия', 8),
 ('европа', 7),
 ('ао', 6),
 ('ра', 6),
 ('правление', 5),
 ('мсфо', 5),
 ('нпф', 5)]

#### Выше представлен один из способов выделения ключевых сущностей